In [17]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import warnings
import sklearn
import sklearn.model_selection
np.set_printoptions(precision=3, suppress=True)  # Print as 0.001 instead of 9.876e-4
torch.set_printoptions(precision=3, sci_mode=False)



In [18]:
#adding match data from file
data = np.loadtxt('newmatchdata.csv',skiprows=1,delimiter=',')
X_data = np.array(data[:,1:]).astype(np.int32)
y_data = np.array(data[:,:1]).astype(np.int32)

#turn all data into feature vector
#feature vector creation
X_val_trn = torch.zeros((40000,138*2),dtype=torch.float32)
j = 0
for d in X_data:
    for i in range(len(d)):
        if(i < len(d)/2):
            h = d[i]
            X_val_trn[j][h] += 1
            
            
        else:
            h = d[i]
            X_val_trn[j][h+137] += 1
      
    j += 1



In [19]:
#sklearn train test split
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_val_trn,y_data, test_size = 0.2, shuffle= True)

#split train into train/validation set

X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X_train,y_train, test_size = 0.15, shuffle= True)

X_train = torch.tensor(X_train,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32)

X_val = torch.tensor(X_val,dtype=torch.float32)

y_val = torch.tensor(y_val,dtype=torch.float32)

X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.float32)
#sklearn train test split
#use test set to calculate error, CCE, accuracy etc. 

/var/folders/w2/kkshvcxn49l4c5_1vv9cd2nr0000gn/T/ipykernel_67867/2977093579.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train,dtype=torch.float32)
/var/folders/w2/kkshvcxn49l4c5_1vv9cd2nr0000gn/T/ipykernel_67867/2977093579.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_val,dtype=torch.float32)
/var/folders/w2/kkshvcxn49l4c5_1vv9cd2nr0000gn/T/ipykernel_67867/2977093579.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test,dtype=torch.float32)


In [105]:

#model creation
#addition of multiple hidden layers and drop rate to help with regularization
model = torch.nn.Sequential(
    torch.nn.Linear(276,100),
    torch.nn.Dropout(),
    torch.nn.ReLU(),
    torch.nn.Linear(100,50),
    torch.nn.Dropout(),
    torch.nn.ReLU(),
    torch.nn.Linear(50,1)
)

model2 = torch.nn.Sequential(
    torch.nn.Linear(276,1)
)

In [32]:
model2 = torch.nn.Sequential(
    torch.nn.Linear(276,138),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.1),
    torch.nn.Linear(138,69),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(69,1)
)



#loss function and optimizer
loss = torch.nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(model2.parameters(),lr=5e-5, weight_decay=2.5e-5)

#epoch number
num_epoch = 120
next_epoch = 1
batch_size = 80

#training loop
for epoch in range(next_epoch, next_epoch+num_epoch):
    
    
    # Make an entire pass (an 'epoch') over the training data in batch_size chunks
    for i in range(0, len(X_train), batch_size):        
        X = X_train[i:i+batch_size]     # Slice out a mini-batch of features
        y = y_train[i:i+batch_size]     # Slice out a mini-batch of targets
        
        y_pred = model2(X)                   # Make predictions (final-layer activations)
        
        l = loss(y_pred, y)                 # Compute loss with respect to predictions
        
        model2.zero_grad()                   # Reset all gradient accumulators to zero (PyTorch thing)
        l.backward()                        # Compute gradient of loss wrt all parameters (backprop!)
        optim.step()                    # Use the gradients to take a step with SGD.
        
    print("Epoch %2d: loss on final training batch: %.4f" % (epoch, l.item()))
    # pred = torch.sign(model(X_val))
    # acc = torch.mean((pred == y_val).float())
    
    #create dire query

    #validation set calculations
    dire_X =  torch.index_select(X_val, 1, torch.LongTensor([*range(138,276)]))
    dire_X = torch.cat((dire_X,torch.index_select(X_val, 1, torch.LongTensor([*range(0,138)]))),1)

    dire_pred = (model2(dire_X) >= 0).float()
    
    rad_pred = (model2(X_val) >= 0).float()
    

    overall_prob = (((rad_pred + (1 - dire_pred))/2) > 0.5).float()
    
    val_acc = torch.mean((overall_prob == y_val).float())

    #training set calculations
    dire_X_train =  torch.index_select(X_train, 1, torch.LongTensor([*range(138,276)]))
    dire_X_train = torch.cat((dire_X_train,torch.index_select(X_train, 1, torch.LongTensor([*range(0,138)]))),1)

    dire_pred = (model2(dire_X_train) >= 0).float()
    
    rad_pred = (model2(X_train) >= 0).float()
    

    overall_prob = (((rad_pred + (1 - dire_pred))/2) > 0.5).float()
    
    train_acc = torch.mean((overall_prob == y_train).float())
    
    print("Epoch %2d: accuracy on validation set: %.4f" % (epoch, val_acc))
    print("Epoch %2d: accuracy on training set: %.4f" % (epoch, train_acc))
    
    print("Epoch %2d: loss on validation set: %.4f" % (epoch, loss(model2(X_val), y_val)))
    
   

next_epoch = epoch+1

Epoch  1: loss on final training batch: 0.6910
Epoch  1: accuracy on validation set: 0.4963
Epoch  1: accuracy on training set: 0.4919
Epoch  1: loss on validation set: 0.6930
Epoch  2: loss on final training batch: 0.6926
Epoch  2: accuracy on validation set: 0.4958
Epoch  2: accuracy on training set: 0.4927
Epoch  2: loss on validation set: 0.6923
Epoch  3: loss on final training batch: 0.6908
Epoch  3: accuracy on validation set: 0.5038
Epoch  3: accuracy on training set: 0.4992
Epoch  3: loss on validation set: 0.6916
Epoch  4: loss on final training batch: 0.6890
Epoch  4: accuracy on validation set: 0.5154
Epoch  4: accuracy on training set: 0.5135
Epoch  4: loss on validation set: 0.6899
Epoch  5: loss on final training batch: 0.6862
Epoch  5: accuracy on validation set: 0.5183
Epoch  5: accuracy on training set: 0.5247
Epoch  5: loss on validation set: 0.6871
Epoch  6: loss on final training batch: 0.6862
Epoch  6: accuracy on validation set: 0.5240
Epoch  6: accuracy on traini

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_val_trn,y_data, test_size = 0.2, shuffle= True)

mlp = MLPClassifier(hidden_layer_sizes = 4,activation='relu',solver='adam',batch_size=16,learning_rate_init=0.05,max_iter=3000,random_state=0)# Your code here
log = LogisticRegression()
mlp.fit(X_train,y_train.ravel())
log.fit(X_train,y_train.ravel())

print('training accuracy: ',mlp.score(X_train,y_train)*100,'%')
print('held-out accuracy (testing): ',mlp.score(X_test,y_test)*100,'%')

dire_X =  torch.index_select(X_test, 1, torch.LongTensor([*range(138,276)]))
dire_X = torch.cat((dire_X, torch.index_select(X_test, 1, torch.LongTensor([*range(0,138)]))),1)



dire_prob = log.score(dire_X,y_test)
rad_prob = log.score(X_test,y_test)

overall_prob = ((rad_prob + (1 - dire_prob))/2)

print('log training accuracy: ',log.score(X_train,y_train)*100,'%')
print('log held-out accuracy (testing): ',log.score(X_test,y_test)*100,'%')

print('log overall accuracy: ',overall_prob*100,'%')

for k in range(2,20):
    scores = sklearn.model_selection.cross_val_score(log,X_test,y_test.ravel(),cv=k) 
    print("held-out accuracy (%d-fold):   %.1f%%" % (k, scores.mean()*100))


training accuracy:  49.471875 %
held-out accuracy (testing):  48.5625 %
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


SyntaxError: 'break' outside loop (226846857.py, line 18)

In [33]:
#test accuracy on test set 
dire_X =  torch.index_select(X_test, 1, torch.LongTensor([*range(138,276)]))
dire_X = torch.cat((dire_X,torch.index_select(X_test, 1, torch.LongTensor([*range(0,138)]))),1)
dire_pred = (model2(dire_X) >= 0).float()
rad_pred = (model2(X_test) >= 0).float()


overall_prob = (((rad_pred + (1 - dire_pred))/2) > 0.5).float()
acc = torch.mean((overall_prob == y_test).float())
print(acc)

tensor(0.511)
